In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Funzione per calcolare la distanza euclidea

In [29]:
def euclideanDistance(ex, query):
    distance = np.linalg.norm(ex - query)
    return distance

Visto che peseremo i risultati in base alla distanza serve una funzione per definire il peso

In [30]:
def weightByDistance(distance):
    weight = 1/distance
    return weight

K-NN pesato

In [31]:
'''def kNN_weighted(examples, query, k):
    class_distance = []
    for i in range(examples.shape[0]):
        classe = examples.at[i, 'hazardous']
        single_example = examples.loc[i].drop('hazardous').values
        distance = euclideanDistance(single_example, query.values)
        if distance == 0:
            return classe
        class_distance.append((distance, classe))

    class_distance = sorted(class_distance)[:k]
    freq_hazardous = 0
    freq_not_hazardous = 0

    for example in class_distance:
        if example[1] == 'True':
            freq_hazardous = freq_hazardous + weightByDistance(example[0])
        else:
            freq_not_hazardous = freq_not_hazardous + weightByDistance(example[0])

    if freq_hazardous  > freq_not_hazardous :
        return 'True'
    else:
        return 'False'
        '''


def kNN_weighted(examples, queries, k):
    distances = np.linalg.norm(examples.drop('hazardous', axis=1).values[:, np.newaxis, :] - queries.values, axis=2)
    
    class_distance = [(distances[i], examples.at[i, 'hazardous']) for i in range(examples.shape[0])]
    class_distance = sorted(class_distance, key=lambda x: x[0])[:k]
    
    freq_hazardous = sum(weightByDistance(example[0]) for example in class_distance if example[1] == 'True')
    freq_not_hazardous = sum(weightByDistance(example[0]) for example in class_distance if example[1] == 'False')
    
    if freq_hazardous > freq_not_hazardous:
        return 'True'
    else:
        return 'False'


Matrice di confusione

In [32]:
def confusionMatrix(classi):
    # True hazardous
    TH = classi.loc[(classi['hazardous'] == 'True') & (classi['hazardous_predicted'] == 'True')].reset_index(drop=True).shape[0] 
    # False hazardous
    FH = classi.loc[(classi['hazardous'] == 'False') & (classi['hazardous_predicted'] == 'True')].reset_index(drop=True).shape[0]
    # False not hazardous
    FNH = classi.loc[(classi['hazardous'] == 'True') & (classi['hazardous_predicted'] == 'False')].reset_index(drop=True).shape[0]
    # True not hazardous
    TNH = classi.loc[(classi['hazardous'] == 'False') & (classi['hazardous_predicted'] == 'False')].reset_index(drop=True).shape[0]

    conf_matrix = pd.DataFrame({"hazardous": [TH, FH], "not hazardous": [FNH, TNH]}, index = ["hazardous", "not hazardous"])

    return(conf_matrix, TH, FH, FNH, TNH)

Valutazione

In [33]:
def evaluation(TH, FH, FNH, TNH):
    m = TH + FH + FNH + TNH

    # classificazioni corrette
    correct_classifications = TH + TNH
    accuracy = correct_classifications / m
    accuracy_percentage = accuracy * 100
    print("Accuratezza del modello (istanze classificate correttamente) => " + str(round(accuracy_percentage, 5)) + "\nN° classificazioni corrette =>" + str(correct_classifications))

    #classificazioni errate
    incorrect_classifications = FH + FNH
    inaccuracy = incorrect_classifications / m
    inaccuracy_percentage = inaccuracy * 100
    print("Imprecisione del modello (istanze classificate incorrettamente) => " + str(round(accuracy_percentage, 5)) + "\nN° classificazioni sbagliate =>" + str(correct_classifications))

    #coeff Kappa
    p_true = ((TH + FNH)/m) * ((TNH + FH)/m)
    p_false = ((TNH + FH)/m) * ((TNH + FNH)/m)
    p_e = p_true + p_false
    k_coeff = (accuracy - p_e) / (1 - p_e)
    print("Kappa coefficient => " + str(round(k_coeff, 5)))

    print("\n######## STATISTICHE DELLE DUE CLASSI ########")
    print("\n++++++++ HAZARDOUS ++++++++")
    precision = TH / (TH + FH)
    print("Precisione = " + str(round(precision, 5)))
    recall = TH / (TH + FNH)
    print("Recall = " + str(round(recall, 5)))
    F1_score = 2 * ((precision * recall) / (precision + recall))
    print("F1 score = " + str(round(F1_score, 5)))

    print("\n-------- NOT HAZARDOUS -------")
    precision = TNH / (TNH + FNH)
    print("Precisione = " + str(round(precision, 5)))
    recall = TNH / (TNH + FH)
    print("Recall = " + str(round(recall, 5)))
    F1_score = 2 * ((precision * recall) / (precision + recall))
    print("F1 score = " + str(round(F1_score, 5)))


caricamento del dataset e definizione del numero di vicini

In [34]:
dataset = pd.read_csv('neo_v2_edited.csv')

dataset = dataset.sample(frac=1).reset_index(drop=True)

k=10

In [35]:
print('\n******* TEST MODE: SPLIT 70% TRAIN, REMAINDER TEST *******')

# Splitting training/test set (70% training set e 30% test set)
training_set = dataset.head(round(len(dataset)*(70/100))) # arrotondo per eccesso
test_set = dataset.tail(len(dataset) - len(training_set)).reset_index(drop=True)
test_set['hazardous_predicted'] = np.zeros(test_set.shape[0])    # viene inserita la classe predetta

for i in range(test_set.shape[0]):
    print('Predico query numero ' + str(i))
    query = test_set[i:(i+1)].drop('hazardous', axis = 1).drop('hazardous_predicted', axis = 1).reset_index(drop=True)
    test_set.at[i,'hazardous_predicted'] = kNN_weighted(training_set, query, k)


******* TEST MODE: SPLIT 70% TRAIN, REMAINDER TEST *******
Predico query numero 0
Predico query numero 1
Predico query numero 2
Predico query numero 3
Predico query numero 4
Predico query numero 5
Predico query numero 6
Predico query numero 7
Predico query numero 8
Predico query numero 9
Predico query numero 10
Predico query numero 11
Predico query numero 12
Predico query numero 13
Predico query numero 14
Predico query numero 15
Predico query numero 16
Predico query numero 17
Predico query numero 18
Predico query numero 19
Predico query numero 20
Predico query numero 21
Predico query numero 22
Predico query numero 23
Predico query numero 24
Predico query numero 25
Predico query numero 26
Predico query numero 27
Predico query numero 28
Predico query numero 29
Predico query numero 30
Predico query numero 31
Predico query numero 32
Predico query numero 33
Predico query numero 34
Predico query numero 35
Predico query numero 36
Predico query numero 37
Predico query numero 38
Predico query 

Valutazione

In [ ]:
print('\n------- MATRICE DI CONFUSIONE -------')
classi = test_set[['hazardous','hazardous_predicted']]
(confusione, TH, FH, FNH, TNH) = confusionMatrix(classi)
print(confusione)

evaluation(TH, FH, FNH, TNH)